## Libraries

In [ ]:
import io
import math
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import re
import time
import numpy as np
import seaborn as sns
import pickle
import datetime
from scipy.stats import linregress
from matplotlib.pyplot import figure
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Labelling Splitting**

In [ ]:
label = pd.read_csv("/content/drive/MyDrive/TA/Label.csv")

In [ ]:
label.shape

(1860, 26)

In [ ]:
label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   BLOCK                      1860 non-null   object 
 1   FIELD                      1860 non-null   object 
 2   STATION/ PLANT             1860 non-null   object 
 3   PRODUCTION DATE            1860 non-null   object 
 4   WELL                       1860 non-null   object 
 5   CLUSTER                    1860 non-null   object 
 6   METHOD                     1860 non-null   object 
 7   TYPE                       1860 non-null   object 
 8   RF OIL                     1860 non-null   float64
 9   RF GAS                     1860 non-null   float64
 10  RF WAT                     1860 non-null   float64
 11  IMPURITIES RATIO <br> (%)  1860 non-null   int64  
 12  DOWN TIME                  1860 non-null   object 
 13  UP TIME                    1860 non-null   objec

In [ ]:
label[label.isna().any(axis=1)] #check data null

,BLOCK,FIELD,STATION/ PLANT,PRODUCTION DATE,WELL,CLUSTER,METHOD,TYPE,RF OIL,RF GAS,...,GAS POTENTIAL \n (MSCFD),LIQUID POTENTIAL \n (BLPD),LPO OIL \n (BOPD),LPO GAS \n (MSCFD),LPO LIQUID \n (BLPD),LPO LEVEL-1,LPO LEVEL-2,LPO LEVEL-3,USER REMARKS,PLANNED/ UNPLANNED


In [ ]:
label=label.dropna()

In [ ]:
label=label.drop(columns=['BLOCK', 'FIELD', 'STATION/ PLANT',
       'CLUSTER', 'METHOD', 'TYPE', 'RF OIL', 'RF GAS', 'RF WAT',
       'IMPURITIES RATIO <br> (%)', 'DOWN TIME', 'UP TIME', 'LPO DURATION',
       'OIL POTENTIAL \n (BOPD)', 'GAS POTENTIAL \n (MSCFD)',
       'LIQUID POTENTIAL \n (BLPD)', 'LPO OIL \n (BOPD)', 'LPO GAS \n (MSCFD)',
       'LPO LIQUID \n (BLPD)', 'LPO LEVEL-1', 'LPO LEVEL-2', 'LPO LEVEL-3',
       'PLANNED/ UNPLANNED'])
label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860 entries, 0 to 1859
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCTION DATE  1860 non-null   object
 1   WELL             1860 non-null   object
 2   USER REMARKS     1860 non-null   object
dtypes: object(3)
memory usage: 43.7+ KB


In [ ]:
label.rename(columns={'PRODUCTION DATE':'RecordTime'}, inplace=True)
label.rename(columns={'RecordTime':'DM'}, inplace=True)
label.rename(columns={'WELL':'Wellname'}, inplace=True)

## Setup

In [ ]:
A = pd.read_csv("/content/drive/MyDrive/TA/Tugas Akhir Banget.csv")
A

,Wellname,Well Status (ESP),Motor Ampere,Frequency,Pump Intake Pressure,Temperature Motor,Output Volt,RecordTime,Pump Discharge Pressure,Input Voltage,ESP Type,ESP Stages,Motor Horse Power (HP)
0,KS-0002,1.0,22.4,60.0,0.0,0.0,436.1,2020-10-01 00:09:00,0.0,436.0,DN1750,86.0,60.0
1,KS-0002,1.0,22.4,60.0,0.0,0.0,436.3,2020-10-01 00:19:00,0.0,436.0,DN1750,86.0,60.0
2,KS-0002,1.0,22.3,60.0,0.0,0.0,436.4,2020-10-01 00:29:00,0.0,436.0,DN1750,86.0,60.0
3,KS-0002,1.0,22.4,60.0,0.0,0.0,436.1,2020-10-01 00:39:00,0.0,436.0,DN1750,86.0,60.0
4,KS-0002,1.0,22.4,60.0,0.0,0.0,436.4,2020-10-01 00:49:00,0.0,436.0,DN1750,86.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9832032,KS-0370,1.0,41.0,49.9,0.0,0.0,392.0,2022-08-31,0.0,392.0,P16 SND,94.0,102.0
9832033,KS-0370,1.0,41.0,49.7,0.0,0.0,390.0,2022-08-31,0.0,390.0,P16 SND,94.0,102.0
9832034,KS-0370,1.0,42.0,49.8,0.0,0.0,391.0,2022-08-31,0.0,391.0,P16 SND,94.0,102.0
9832035,KS-0370,1.0,41.0,49.9,0.0,0.0,391.0,2022-08-31,0.0,391.0,P16 SND,94.0,102.0


In [ ]:
df = A

columns = ['Wellname', 'Well Status (ESP)', 'Motor Ampere', 'Frequency',
       'Pump Intake Pressure', 'Temperature Motor', 'Output Volt',
       'RecordTime', 'Pump Discharge Pressure', 'Input Voltage',
       'ESP Stages', 'Motor Horse Power (HP)']
df = df.reindex(columns=columns)
# Unique machine names
unique_machines = df['Wellname'].unique()

# Loop over unique machine names
for machine in unique_machines:
    machine_data = df[df['Wellname'] == machine].copy()  # Filter data for current machine

        # Perform the preparation steps for the current machine
    list_DM = [i.split(" ")[0][0:10] for i in machine_data['RecordTime']]
    list_Year = [i.split(" ")[0][0:4] for i in machine_data['RecordTime']]
    list_Month = [i.split(" ")[0][5:7] for i in machine_data['RecordTime']]
    list_Day = [i.split(" ")[0][8:10] for i in machine_data['RecordTime']]

    machine_data['DM'] = list_DM
    machine_data['Year'] = list_Year
    machine_data['Month'] = list_Month
    machine_data['Day'] = list_Day
    #machine_data = machine_data.groupby(['DM', 'Year', 'Month', 'Day']).mean()
    df.loc[machine_data.index, ['DM', 'Year', 'Month', 'Day']] = machine_data[['DM', 'Year', 'Month', 'Day']]

df

,Wellname,Well Status (ESP),Motor Ampere,Frequency,Pump Intake Pressure,Temperature Motor,Output Volt,RecordTime,Pump Discharge Pressure,Input Voltage,ESP Stages,Motor Horse Power (HP),DM,Year,Month,Day
0,KS-0002,1.0,22.4,60.0,0.0,0.0,436.1,2020-10-01 00:09:00,0.0,436.0,86.0,60.0,2020-10-01,2020,10,01
1,KS-0002,1.0,22.4,60.0,0.0,0.0,436.3,2020-10-01 00:19:00,0.0,436.0,86.0,60.0,2020-10-01,2020,10,01
2,KS-0002,1.0,22.3,60.0,0.0,0.0,436.4,2020-10-01 00:29:00,0.0,436.0,86.0,60.0,2020-10-01,2020,10,01
3,KS-0002,1.0,22.4,60.0,0.0,0.0,436.1,2020-10-01 00:39:00,0.0,436.0,86.0,60.0,2020-10-01,2020,10,01
4,KS-0002,1.0,22.4,60.0,0.0,0.0,436.4,2020-10-01 00:49:00,0.0,436.0,86.0,60.0,2020-10-01,2020,10,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9832032,KS-0370,1.0,41.0,49.9,0.0,0.0,392.0,2022-08-31,0.0,392.0,94.0,102.0,2022-08-31,2022,08,31
9832033,KS-0370,1.0,41.0,49.7,0.0,0.0,390.0,2022-08-31,0.0,390.0,94.0,102.0,2022-08-31,2022,08,31
9832034,KS-0370,1.0,42.0,49.8,0.0,0.0,391.0,2022-08-31,0.0,391.0,94.0,102.0,2022-08-31,2022,08,31
9832035,KS-0370,1.0,41.0,49.9,0.0,0.0,391.0,2022-08-31,0.0,391.0,94.0,102.0,2022-08-31,2022,08,31


In [ ]:
B = df
B = B.drop(columns=['Year',	'Month',	'Day', 'Well Status (ESP)','ESP Stages','RecordTime'])
B

,Wellname,Motor Ampere,Frequency,Pump Intake Pressure,Temperature Motor,Output Volt,Pump Discharge Pressure,Input Voltage,Motor Horse Power (HP),DM
0,KS-0002,22.4,60.0,0.0,0.0,436.1,0.0,436.0,60.0,2020-10-01
1,KS-0002,22.4,60.0,0.0,0.0,436.3,0.0,436.0,60.0,2020-10-01
2,KS-0002,22.3,60.0,0.0,0.0,436.4,0.0,436.0,60.0,2020-10-01
3,KS-0002,22.4,60.0,0.0,0.0,436.1,0.0,436.0,60.0,2020-10-01
4,KS-0002,22.4,60.0,0.0,0.0,436.4,0.0,436.0,60.0,2020-10-01
...,...,...,...,...,...,...,...,...,...,...
9832032,KS-0370,41.0,49.9,0.0,0.0,392.0,0.0,392.0,102.0,2022-08-31
9832033,KS-0370,41.0,49.7,0.0,0.0,390.0,0.0,390.0,102.0,2022-08-31
9832034,KS-0370,42.0,49.8,0.0,0.0,391.0,0.0,391.0,102.0,2022-08-31
9832035,KS-0370,41.0,49.9,0.0,0.0,391.0,0.0,391.0,102.0,2022-08-31


In [ ]:
C = B
columns_to_aggregate = [
    'Motor Ampere', 'Frequency', 'Pump Intake Pressure',
    'Temperature Motor', 'Output Volt', 'Pump Discharge Pressure',
    'Input Voltage', 'Motor Horse Power (HP)'
]

result_list = []

# Iterate over each unique Wellname
for wellname in C['Wellname'].unique():
    # Subset the DataFrame for the current Wellname
    subset_df = C[C['Wellname'] == wellname]

    # Perform the groupby operation
    grouped_df = subset_df.groupby('DM')[columns_to_aggregate].mean().reset_index()

    # Add a Wellname column to the grouped result
    grouped_df['Wellname'] = wellname

    # Append the result to the list
    result_list.append(grouped_df)

# Concatenate all the results
final_df = pd.concat(result_list).reset_index(drop=True)



In [ ]:
final_df

,DM,Motor Ampere,Frequency,Pump Intake Pressure,Temperature Motor,Output Volt,Pump Discharge Pressure,Input Voltage,Motor Horse Power (HP),Wellname
0,2020-10-01,22.335238,60.0,0.0,0.0,435.960952,0.0,436.000000,60.0,KS-0002
1,2020-10-02,22.325962,60.0,0.0,0.0,436.136538,0.0,436.000000,60.0,KS-0002
2,2020-10-03,22.294574,60.0,0.0,0.0,437.184496,0.0,436.000000,60.0,KS-0002
3,2020-10-04,22.276230,60.0,0.0,0.0,437.315574,0.0,436.000000,60.0,KS-0002
4,2020-10-05,22.319014,60.0,0.0,0.0,436.442254,0.0,436.000000,60.0,KS-0002
...,...,...,...,...,...,...,...,...,...,...
39368,2022-08-27,21.221831,48.0,0.0,32.0,384.000000,0.0,462.802817,60.0,KS-0188
39369,2022-08-28,21.208856,48.0,0.0,32.0,384.000000,0.0,463.188192,60.0,KS-0188
39370,2022-08-29,21.181119,48.0,0.0,32.0,384.000000,0.0,463.150350,60.0,KS-0188
39371,2022-08-30,21.147687,48.0,0.0,32.0,384.000000,0.0,462.822064,60.0,KS-0188


In [ ]:
merged_df = pd.merge(final_df, label, on=['DM', 'Wellname'], how='left')
merged_df['USER REMARKS']=merged_df['USER REMARKS'].fillna('Normal')
merged_df

,DM,Motor Ampere,Frequency,Pump Intake Pressure,Temperature Motor,Output Volt,Pump Discharge Pressure,Input Voltage,Motor Horse Power (HP),Wellname,USER REMARKS
0,2020-10-01,22.335238,60.0,0.0,0.0,435.960952,0.0,436.000000,60.0,KS-0002,Normal
1,2020-10-02,22.325962,60.0,0.0,0.0,436.136538,0.0,436.000000,60.0,KS-0002,Normal
2,2020-10-03,22.294574,60.0,0.0,0.0,437.184496,0.0,436.000000,60.0,KS-0002,Normal
3,2020-10-04,22.276230,60.0,0.0,0.0,437.315574,0.0,436.000000,60.0,KS-0002,Normal
4,2020-10-05,22.319014,60.0,0.0,0.0,436.442254,0.0,436.000000,60.0,KS-0002,Normal
...,...,...,...,...,...,...,...,...,...,...,...
39372,2022-08-27,21.221831,48.0,0.0,32.0,384.000000,0.0,462.802817,60.0,KS-0188,Normal
39373,2022-08-28,21.208856,48.0,0.0,32.0,384.000000,0.0,463.188192,60.0,KS-0188,Normal
39374,2022-08-29,21.181119,48.0,0.0,32.0,384.000000,0.0,463.150350,60.0,KS-0188,Normal
39375,2022-08-30,21.147687,48.0,0.0,32.0,384.000000,0.0,462.822064,60.0,KS-0188,Normal


##  RUL Counting

In [ ]:
d = merged_df
d['EST RUL'] = 0

# Find the index of the first failure event
failure_indices = d[d['USER REMARKS'] != 'Normal'].index

# Create a new column 'EST RUL' and initialize it to 0
d['EST RUL'] = 0

# Iterate over the rows and calculate estimated RUL
for idx, row in d.iterrows():
    if failure_indices.size == 0:
        d.at[idx, 'EST RUL'] = d.shape[0] - idx - 1
    else:
        next_failure_index = failure_indices[0]
        if idx < next_failure_index:
            d.at[idx, 'EST RUL'] = next_failure_index - idx
        else:
            failure_indices = failure_indices[1:]
            if failure_indices.size > 0:
                next_failure_index = failure_indices[0]
            else:
                next_failure_index = d.shape[0]
            d.at[idx, 'EST RUL'] = next_failure_index - idx
d

,DM,Motor Ampere,Frequency,Pump Intake Pressure,Temperature Motor,Output Volt,Pump Discharge Pressure,Input Voltage,Motor Horse Power (HP),Wellname,USER REMARKS,EST RUL
0,2020-10-01,22.335238,60.0,0.0,0.0,435.960952,0.0,436.000000,60.0,KS-0002,Normal,258
1,2020-10-02,22.325962,60.0,0.0,0.0,436.136538,0.0,436.000000,60.0,KS-0002,Normal,257
2,2020-10-03,22.294574,60.0,0.0,0.0,437.184496,0.0,436.000000,60.0,KS-0002,Normal,256
3,2020-10-04,22.276230,60.0,0.0,0.0,437.315574,0.0,436.000000,60.0,KS-0002,Normal,255
4,2020-10-05,22.319014,60.0,0.0,0.0,436.442254,0.0,436.000000,60.0,KS-0002,Normal,254
...,...,...,...,...,...,...,...,...,...,...,...,...
39372,2022-08-27,21.221831,48.0,0.0,32.0,384.000000,0.0,462.802817,60.0,KS-0188,Normal,4
39373,2022-08-28,21.208856,48.0,0.0,32.0,384.000000,0.0,463.188192,60.0,KS-0188,Normal,3
39374,2022-08-29,21.181119,48.0,0.0,32.0,384.000000,0.0,463.150350,60.0,KS-0188,Normal,2
39375,2022-08-30,21.147687,48.0,0.0,32.0,384.000000,0.0,462.822064,60.0,KS-0188,Normal,1


In [ ]:
d['EST RUL'].value_counts()

EST RUL
1       382
2       123
3       119
4       115
5       113
       ... 
3383      1
3384      1
3385      1
3386      1
0         1
Name: count, Length: 3948, dtype: int64

In [ ]:
d.to_csv('/content/drive/MyDrive/TA/NEW DATABASE/THESISML.csv', index=False)

## Scalling

In [ ]:
df_numeric = d.drop(['Wellname', 'USER REMARKS', 'EST RUL', 'DM'], axis=1)

# Normalize features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_numeric)

# Create a DataFrame with scaled features
scaled_df = pd.DataFrame(scaled_features, columns=df_numeric.columns)
scaled_df['EST RUL'] = d['EST RUL']

scaled_df

,Motor Ampere,Frequency,Pump Intake Pressure,Temperature Motor,Output Volt,Pump Discharge Pressure,Input Voltage,Motor Horse Power (HP),EST RUL
0,0.045995,0.652022,0.0,0.000000,0.789402,0.0,0.570158,0.363636,258
1,0.045976,0.652022,0.0,0.000000,0.789720,0.0,0.570158,0.363636,257
2,0.045911,0.652022,0.0,0.000000,0.791618,0.0,0.570158,0.363636,256
3,0.045873,0.652022,0.0,0.000000,0.791855,0.0,0.570158,0.363636,255
4,0.045961,0.652022,0.0,0.000000,0.790274,0.0,0.570158,0.363636,254
...,...,...,...,...,...,...,...,...,...
39372,0.043702,0.521617,0.0,0.101647,0.695316,0.0,0.605208,0.363636,4
39373,0.043675,0.521617,0.0,0.101647,0.695316,0.0,0.605712,0.363636,3
39374,0.043618,0.521617,0.0,0.101647,0.695316,0.0,0.605663,0.363636,2
39375,0.043549,0.521617,0.0,0.101647,0.695316,0.0,0.605234,0.363636,1


In [ ]:
new.to_csv('/content/drive/MyDrive/TA/NEW DATABASE/THESISMLOTL.csv', index=False)